## Neural error mitigation

### Get noisy VQE results

In this example we run the noisy VQE wit

In [2]:
try:
    from qiskit_ibm_runtime import QiskitRuntimeService
except:
    %pip install qiskit-ibm-runtime 
from qiskit.providers.fake_provider import FakeMelbourne

backend = FakeMelbourne()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.6/253.6 KB 42.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.8/233.8 KB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 KB 12.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.7/129.7 KB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 35.9 MB/s eta 0:00:00
   

ModuleNotFoundError: No module named 'qiskit'

In [3]:
from VQE_noise import *
from qoai_qiskit import *
provider = load_account()

raw_result_noise, counts_noise, values_noise, params_noise, ansatz, vqe, noise_model, sampler, qubit_op = run_VQE(main_chain='APRLRFY', side_chains=None, 
                            interaction='MJ',penalty_back=10,penalty_chiral=10,penalty_1=10,
                            fake_backend = backend,# noise_model='depolarizing',p=0.01,
                            resilience_level=1, aggregation=0.1)

ModuleNotFoundError: No module named 'qiskit_research'

In [0]:
print(raw_result_noise)
raw_result_noise

In [0]:
with open('results/APRLRFY/vqe_state_trex_fake.npy', 'wb') as f:
    pickle.dump(raw_result_noise,f)

### Get brute-force ground state

For comparison, we test all the possible bit-strings and sort them according to the energy of the system.

In [0]:
idx, energies = get_ground_state(qubit_op)
print('Sorted bit-srings (in decimal form) ',idx[:5], ' with energies ', energies[:5])

### Get the measurements data

From the best VQE parameters run 3000 simulations and get the samples in each case (i.e in shot 1 we get '000110010', in shot 2 we get '111000101', etc.) We perform measurements in multiple basis, where each qubit is measured in the Z basis and 0,1 or 2 qubits are measured in the X basis (i.e 'ZZZZZZZZZ', 'XZZZZZZZZ', 'XXZZZZZZZ',...)

In [0]:
measurement_data, bases, dists = get_measurement_dict(ansatz, params_noise, sampler, num_samples_per_basis=3000)

### Get neural quantum state network

Generate the transformer neural network which produces quantum state tomography from a small set of measurements.

In [0]:
import nqs_models
from trainers import NeuralErrorMitigationTrainer
import physics_models
import torch
print ("Is GPU cuda device available = ", torch.cuda.is_available())
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print ("Module using ", device, "device" )
cpu = torch.device('cpu')

# Physics model
mol = physics_models.ProteinFoldingModel(qubit_op)
nqs_transformer = nqs_models.TransformerWF( num_sites = ansatz.num_qubits,
                                            num_layers=2,
                                            internal_dimension=8,
                                            num_heads=4,
                                            dropout=0.0).to(device)

### Train the neural error mitigation algorithm

the algorithm has two steps: the neural quantum state, which learns the quantum statevector given the set of measurements, and the Variational Monte Carlo algorithm, which refines the solution. 

In [0]:
epsilon = 0.0
vmc_iters = 1000
constant_reg_schedule = lambda iter : epsilon if iter < vmc_iters//2 else 0.0
nemtrainer = NeuralErrorMitigationTrainer(nqs_model=nqs_transformer,
                                            measurement_data=measurement_data,
                                            physics_model=mol,
                                            #NQST Parameters
                                            nqst_max_epochs=10,
                                            nqst_lr=1e-2,
                                            nqst_batch_size=256,
                                            #VMC Parameters
                                            vmc_lr=1e-2,
                                            vmc_iterations=vmc_iters,
                                            vmc_batch_size=2048,
                                            vmc_epsilon=constant_reg_schedule,
                                            vmc_regularization_type='L1',
                                            logdir=None
                                            )

nemtrainer.train()

### Visualize the results

In [0]:
import numpy as np
import matplotlib.pyplot as plt
final_state = nemtrainer.final_errmit_state.full_state().detach().numpy() 
distribution = (final_state*final_state.conj()).real

tomography_state = nemtrainer.final_nqst_state.full_state().detach().numpy() 
distribution_tomography = (tomography_state*tomography_state.conj()).real
print('Ground state before error mitigation: ',raw_result_noise.best_measurement['state'], ' with probability ',  raw_result_noise.best_measurement['probability'])

print('Ground state after error mitigation: ', np.arange(distribution.shape[0])[distribution>0.1][0], ' with probability ', distribution[distribution>0.1][0])

fig, (ax1) = plt.subplots(1,1,figsize=(10,5))
ln1 = ax1.plot(range(distribution.shape[0]), distribution, color='red', label="Error mitigated distribution")
ax1.set_ylim(0,1.05)
ax1.set_xlabel("Basis index")
ax1.set_ylabel("Probability")
ax2 = ax1.twinx()
ln2 = ax2.bar(list(raw_result_noise.eigenstate.keys()), list(raw_result_noise.eigenstate.values()), label="Solution from noisy VQE")
ax2.set_ylim(0,0.2)
ax2.set_ylabel("Probability")
#ln3 = ax2.plot(range(distribution_tomography.shape[0]), distribution_tomography, color='green')
plt.legend([ln1[0], ln2, ln3[0]], ['Error mitigated distribution', 'Solution from noisy VQE',
            'Distribution tomography'], loc='upper left')

### Save results to files

In [0]:
import pickle
with open('results/APRLRFY/error_mitigated_state_fake.npy', 'wb') as f:
    np.save(f,final_state)

with open('results/APRLRFY/vqe_state_fake.npy', 'wb') as f:
    pickle.dump(raw_result_noise,f)

## Load results

In [0]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
with open('results/APRLRFY/error_mitigated_state_depolarizing01.npy', 'rb') as f:
    state_mitig_01 = np.load(f)
with open('results/APRLRFY/vqe_state_depolarizing01.npy', 'rb') as f:
    state_noise_01 = pickle.load(f)

with open('results/APRLRFY/error_mitigated_state_depolarizing05.npy', 'rb') as f:
    state_mitig_05 = np.load(f)
with open('results/APRLRFY/vqe_state_depolarizing05.npy', 'rb') as f:
    state_noise_05 = pickle.load(f)

with open('results/APRLRFY/error_mitigated_state_depolarizing1.npy', 'rb') as f:
    state_mitig_1 = np.load(f)
with open('results/APRLRFY/vqe_state_depolarizing1.npy', 'rb') as f:
    state_noise_1 = pickle.load(f)

with open('results/APRLRFY/error_mitigated_state_depolarizing2.npy', 'rb') as f:
    state_mitig_2 = np.load(f)
with open('results/APRLRFY/vqe_state_depolarizing2.npy', 'rb') as f:
    state_noise_2 = pickle.load(f)

with open('results/APRLRFY/error_mitigated_state_fake.npy', 'rb') as f:
    state_mitig_fake = np.load(f)
with open('results/APRLRFY/vqe_state_fake.npy', 'rb') as f:
    state_noise_fake = pickle.load(f)

states_mitig_list = [state_mitig_01,state_mitig_05,state_mitig_1,state_mitig_2, state_mitig_fake]
states_noise_list = [state_noise_01,state_noise_05,state_noise_1,state_noise_2,state_noise_fake]
titles = ['Depolarizing p=0.01', 'Depolarizing p=0.05', 'Depolarizing p=0.1', 'Depolarizing p=0.2',
            'Fake backend']

## Neural Error Mitigation Results

In [0]:
row,col=3,2
fig, ax = plt.subplots(row,col,figsize=(10,10))
for k in range(len(states_mitig_list)):
    i = k//col
    j = k%col
    mitig = states_mitig_list[k]
    mitig = (mitig*mitig.conj()).real
    noise = states_noise_list[k]
    
    ln1 = ax[i][j].plot(range(mitig.shape[0]), mitig, color='red')
    ax[i][j].set_ylim(0,1.05)
    ax2 = ax[i][j].twinx()
    ln3 = ax2.axvline(165, color='green', alpha=0.5)
    ax2.axvline(197, color='green', alpha=0.5)
    ln4 = ax2.axvline(322, color='orange', alpha=0.5)
    ln2 = ax2.bar(list(noise.eigenstate.keys()), list(noise.eigenstate.values()))
    ax2.set_ylim(0,0.2)
    ax2.set_title(titles[k])
    
    if k==0:
        ax2.set_ylabel("Probability")
        fig.legend([ln1[0], ln2, ln3, ln4], ['Error mitigated distribution', 'Solution from noisy VQE',
        'True ground states', 'First excited state'], bbox_to_anchor=(0.95,1.05), ncol=4)
    else:
        ax[i][j].set_xticks([])
        ax[i][j].set_yticks([])
        ax2.set_xticks([])
        ax2.set_yticks([])
ax[0][0].set_xlabel("Basis index")
ax[0][0].set_ylabel("Probability")
ax[2][1].remove()
fig.tight_layout(pad=1.5)



## t-REx results

In [0]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
with open('results/APRLRFY/vqe_state_trex_01.npy', 'rb') as f:
    state_mitig_01 = np.load(f, allow_pickle=True)
with open('results/APRLRFY/vqe_state_trex_05.npy', 'rb') as f:
    state_mitig_05 = np.load(f, allow_pickle=True)
with open('results/APRLRFY/vqe_state_trex_1.npy', 'rb') as f:
    state_mitig_1 = np.load(f, allow_pickle=True)
with open('results/APRLRFY/vqe_state_trex_2.npy', 'rb') as f:
    state_mitig_2 = np.load(f, allow_pickle=True)
with open('results/APRLRFY/vqe_state_trex_fake.npy', 'rb') as f:
    state_mitig_fake = np.load(f, allow_pickle=True)

states_mitig_list = [state_mitig_01,state_mitig_05,state_mitig_1,state_mitig_2, state_mitig_fake]
states_noise_list = [state_noise_01,state_noise_05,state_noise_1,state_noise_2,state_noise_fake]
titles = ['Depolarizing p=0.01', 'Depolarizing p=0.05', 'Depolarizing p=0.1', 'Depolarizing p=0.2',
            'Fake backend']

In [0]:
row,col=3,2
fig, ax = plt.subplots(row,col,figsize=(10,10))
for k in range(len(states_mitig_list)):
    i = k//col
    j = k%col
    mitig = states_mitig_list[k]
    noise = states_noise_list[k]
    ln1 = ax[i][j].bar(list(mitig.eigenstate.keys()), list(mitig.eigenstate.values()), color='red')
    ax[i][j].set_ylim(0,0.2)
    ax2 = ax[i][j].twinx()
    ln3 = ax2.axvline(165, color='green', alpha=0.5)
    ax2.axvline(197, color='green', alpha=0.5)
    ln4 = ax2.axvline(322, color='orange', alpha=0.5)
    ln2 = ax2.bar(list(noise.eigenstate.keys()), list(noise.eigenstate.values()))
    ax2.set_ylim(0,0.2)
    ax2.set_title(titles[k])
    
    if k==0:
        ax2.set_ylabel("Probability")
        fig.legend([ln1, ln2, ln3, ln4], ['Error mitigated distribution', 'Solution from noisy VQE',
        'True ground states', 'First excited state'], bbox_to_anchor=(0.95,1.05), ncol=4)
    else:
        ax[i][j].set_xticks([])
        ax[i][j].set_yticks([])
        ax2.set_xticks([])
        ax2.set_yticks([])
ax[0][0].set_xlabel("Basis index")
ax[0][0].set_ylabel("Probability")
ax[2][1].remove()
fig.tight_layout(pad=1.5)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ddc54cc5-de90-4346-ada1-4ff24feba045' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>